In [1]:
# importing pandas for reading and clearing data 
import pandas as pd
# reading the data set 
dataset2 = pd.read_excel("Input.xlsx")

In [3]:
# imported request module for send request to URL 
import requests
# imported BeautifulSoup for web sraping 
from bs4 import BeautifulSoup
# Loop through each URL in the dataset and send a GET request
for index, row in dataset2.iterrows():
    url = row['URL']
    try:
        response = requests.get(url, timeout=15)
        if response.status_code == 200:
            soup = BeautifulSoup(response.text, 'html.parser')
            title = soup.title.string if soup.title else 'No Title'
            content = ' '.join([p.get_text() for p in soup.find_all('div', class_="td-post-content")])
            # Store the title and content as strings
            dataset2.at[index, 'Title'] = str(title)
            dataset2.at[index, 'Content'] = str(content)
    except requests.exceptions.RequestException as e:
        print(f"Failed to process URL {url}: {e}")

C:\Users\Bavas\AppData\Local\Temp\ipykernel_6868\531140413.py:17: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'Rise of telemedicine and its Impact on Livelihood by 2040 - Blackcoffer Insights' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  dataset2.at[index, 'Title'] = str(title)
C:\Users\Bavas\AppData\Local\Temp\ipykernel_6868\531140413.py:18: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '
Telemedicine, the use of technology to diagnose and treat patients remotely, has been rising in recent years. With the advent of high-speed internet and improved video conferencing tools, healthcare providers are increasingly turning to telemedicine to provide care to patients in remote or underserved areas.Telemedicine, using technology to provide healthcare services remotely, has recently g

In [4]:
# Reading Stopwords provided in text files as well as tokenizing it
import os
directory = 'StopWords/'
contents = []
for filename in os.listdir(directory):
    if filename.endswith(".txt"):
        with open(os.path.join(directory, filename), 'r', encoding='latin-1') as file:
            contents.append(file.read())
combined_content = "".join(contents)
custom_stopwords = combined_content.split('\n')

In [7]:
import os
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
# Download necessary NLTK data
nltk.download('punkt_tab')
# Directory containing your text files
directory = 'MasterDictionary/'
# Initialize lists
positive_words = []
negative_words = []
# Read files and classify words
for filename in os.listdir(directory):
    if filename.endswith(".txt"):
        filepath = os.path.join(directory, filename)
        with open(filepath, 'r', encoding='utf-8', errors='ignore') as file:
            lines = file.readlines()
            for line in lines:
                tokens = word_tokenize(line.strip())  # Tokenizing the line with NLTK
                for word in tokens:
                    # Assuming positive and negative words are labeled within the file content
                    if "positive" in filename.lower():
                        positive_words.append(word)
                    elif "negative" in filename.lower():
                        negative_words.append(word)

# Create a dictionary
sentiment_dict = {
    "positive": positive_words,
    "negative": negative_words
}

[nltk_data] Downloading package punkt_tab to
[nltk_data]     F:\NLP_Project_linkdin_profile\env\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


{'positive': ['a+', 'abound', 'abounds', 'abundance', 'abundant', 'accessable', 'accessible', 'acclaim', 'acclaimed', 'acclamation', 'accolade', 'accolades', 'accommodative', 'accomodative', 'accomplish', 'accomplished', 'accomplishment', 'accomplishments', 'accurate', 'accurately', 'achievable', 'achievement', 'achievements', 'achievible', 'acumen', 'adaptable', 'adaptive', 'adequate', 'adjustable', 'admirable', 'admirably', 'admiration', 'admire', 'admirer', 'admiring', 'admiringly', 'adorable', 'adore', 'adored', 'adorer', 'adoring', 'adoringly', 'adroit', 'adroitly', 'adulate', 'adulation', 'adulatory', 'advanced', 'advantage', 'advantageous', 'advantageously', 'advantages', 'adventuresome', 'adventurous', 'advocate', 'advocated', 'advocates', 'affability', 'affable', 'affably', 'affectation', 'affection', 'affectionate', 'affinity', 'affirm', 'affirmation', 'affirmative', 'affluence', 'affluent', 'afford', 'affordable', 'affordably', 'afordable', 'agile', 'agilely', 'agility', 'ag

In [ ]:
# If we not get data we need to fill nan there
dataset2['Content'] = dataset2['Content'].fillna("")

In [10]:
import pandas as pd
import nltk
import string
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords

# Download necessary NLTK packages
nltk.download('punkt')
nltk.download('stopwords')


# List of personal pronouns to check for
personal_pronouns = {"i", "we", "my", "ours", "us"}

# Function to count syllables in a word
def count_syllables(word):
    word = word.lower().strip(string.punctuation)
    syllables = 0
    vowels = "aeiouy"
    if word and word[0] in vowels:
        syllables += 1
    for index in range(1, len(word)):
        if word[index] in vowels and word[index - 1] not in vowels:
            syllables += 1
    if word.endswith(("es", "ed")):
        syllables -= 1
    if syllables < 1:
        syllables = 1
    return syllables

# Calculate readability and sentiment metrics
def calculate_metrics(text):
    stop_words = set(stopwords.words('english')).union(set(custom_stopwords))
    
    # Tokenize sentences and words
    sentences = sent_tokenize(text)
    words = word_tokenize(text)
    words_no_stop = [word for word in words if word.lower() not in stop_words]
    words_no_stop = [word.strip(string.punctuation) for word in words_no_stop]
    
    # Calculate word counts
    num_words = len(words_no_stop)
    num_sentences = len(sentences)
    num_complex_words = sum(1 for word in words_no_stop if count_syllables(word) > 2)
    total_words = len(words_no_stop)  # After stopword removal
    
    # Calculate syllables per word and avg word length
    syllables_per_word = sum(count_syllables(word) for word in words_no_stop) / total_words if total_words > 0 else 0
    avg_word_length = sum(len(word) for word in words_no_stop) / total_words if total_words > 0 else 0

    # Calculate personal pronoun count
    personal_pronoun_count = sum(1 for word in words if word.lower() in personal_pronouns)

    # Calculate sentiment scores
    positive_score = sum(1 for word in words_no_stop if word.lower() in positive_words)
    negative_score = sum(1 for word in words_no_stop if word.lower() in negative_words)
    
    # Calculate Polarity and Subjectivity
    polarity_score = (positive_score - negative_score) / ((positive_score + negative_score) + 0.000001)
    subjectivity_score = (positive_score + negative_score) / (total_words + 0.000001)
    
    # Calculate readability metrics
    avg_sentence_length = num_words / num_sentences if num_sentences > 0 else 0
    percent_complex_words = num_complex_words / num_words if num_words > 0 else 0
    fog_index = 0.4 * (avg_sentence_length + percent_complex_words)
    
    return (positive_score, negative_score, polarity_score, subjectivity_score, avg_sentence_length, 
            percent_complex_words, fog_index, avg_sentence_length, num_complex_words, total_words, 
            syllables_per_word, personal_pronoun_count, avg_word_length)

# Apply the function to the dataset
dataset2['Positive_Score'], dataset2['Negative_Score'], dataset2['Polarity_Score'], dataset2['Subjectivity_Score'], \
dataset2['Avg_Sentence_Length'], dataset2['Percent_Complex_Words'], dataset2['Fog_Index'], \
dataset2['Avg_Number_Words_Per_Sentence'], dataset2['Complex_Word_Count'], dataset2['Word_Count'], \
dataset2['Syllable_Per_Word'], dataset2['Personal_Pronouns'], dataset2['Avg_Word_Length'] = \
    zip(*dataset2['Content'].apply(calculate_metrics))

# Print the DataFrame to check the result
print(dataset2)


[nltk_data] Downloading package punkt to
[nltk_data]     F:\NLP_Project_linkdin_profile\env\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     F:\NLP_Project_linkdin_profile\env\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


   URL_ID                                                URL  \
0   123.0  https://insights.blackcoffer.com/rise-of-telem...   
1   321.0  https://insights.blackcoffer.com/rise-of-e-hea...   
2  2345.0  https://insights.blackcoffer.com/rise-of-e-hea...   
3  4321.0  https://insights.blackcoffer.com/rise-of-telem...   
4   432.0  https://insights.blackcoffer.com/rise-of-telem...   

                                               Title  \
0  Rise of telemedicine and its Impact on Livelih...   
1  Rise of e-health and its impact on humans by t...   
2  Rise of e-health and its impact on humans by t...   
3  Rise of telemedicine and its Impact on Livelih...   
4  Rise of telemedicine and its Impact on Livelih...   

                                             Content  Avg_Sentence_Length  \
0  \nTelemedicine, the use of technology to diagn...            17.857143   
1  \nThe rise of e-health, or the use of electron...            17.578947   
2  \n2020 was the year the world was ravaged by

C:\Users\Bavas\AppData\Local\Temp\ipykernel_6868\145395356.py:72: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset2['Positive_Score'], dataset2['Negative_Score'], dataset2['Polarity_Score'], dataset2['Subjectivity_Score'], \
C:\Users\Bavas\AppData\Local\Temp\ipykernel_6868\145395356.py:72: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset2['Positive_Score'], dataset2['Negative_Score'], dataset2['Polarity_Score'], dataset2['Subjectivity_Score'], \
C:\Users\Bavas\AppData\Local\Temp\ipykernel_6868\145

In [11]:
dataset2

,URL_ID,URL,Title,Content,Avg_Sentence_Length,Percent_Complex_Words,Fog_Index,Polarity_Score,Subjectivity_Score,Total_Words,Complex_Word_Count,Positive_Score,Negative_Score,Avg_Number_Words_Per_Sentence,Word_Count,Syllable_Per_Word,Personal_Pronouns,Avg_Word_Length
0,123.0,https://insights.blackcoffer.com/rise-of-telem...,Rise of telemedicine and its Impact on Livelih...,"\nTelemedicine, the use of technology to diagn...",17.857143,0.439000,7.318457,0.551020,0.098000,1000,439,76,22,17.857143,1000,2.558000,2,6.788000
1,321.0,https://insights.blackcoffer.com/rise-of-e-hea...,Rise of e-health and its impact on humans by t...,"\nThe rise of e-health, or the use of electron...",17.578947,0.482036,7.224393,0.490196,0.152695,334,161,38,13,17.578947,334,2.514970,3,7.062874
2,2345.0,https://insights.blackcoffer.com/rise-of-e-hea...,Rise of e-health and its impact on humans by t...,\n2020 was the year the world was ravaged by t...,10.650794,0.321908,4.389081,-0.125000,0.071535,671,216,21,27,10.650794,671,2.138599,3,6.041729
3,4321.0,https://insights.blackcoffer.com/rise-of-telem...,Rise of telemedicine and its Impact on Livelih...,"\n“More gains on quality, affordability and ac...",16.782609,0.376943,6.863821,0.114754,0.079016,772,291,34,27,16.782609,772,2.257772,7,6.415803
4,432.0,https://insights.blackcoffer.com/rise-of-telem...,Rise of telemedicine and its Impact on Livelih...,"\n“More gains on quality, affordability and ac...",16.782609,0.376943,6.863821,0.114754,0.079016,772,291,34,27,16.782609,772,2.257772,7,6.415803
